In [7]:
import traceback

import pandas as pd
import tushare as ts

type_max = {'北交所': 29.8, '创业板': 19.8}
# 连续涨停数
grow_up_count = 3

ts.set_token('your token here')
# pro = ts.pro_api()
tu_token = 'fba0f69c02f5e4399b8dd63ae895ca69b7a55122a545c6f0f2c2bef3'
pro = ts.pro_api(tu_token)
start_date = '20220101'


def get_single_grow(ts_code, name, market):
    """
    计算从一个非涨停开始，连续两个涨停之后第三天的涨跌幅
    :param ts_code:
    :return:
    """
    df = pro.daily(ts_code=ts_code, start_date=start_date)
    df_day = pd.concat([df.ts_code, df.trade_date, df.close, df.pct_chg], axis=1)
    df_day.columns = ['ts_code', 'trade_date', 'close', 'grow']
    market_max = type_max.get(market)
    if None is market_max:
        market_max = 9.8
    list_industry = {'股票代码': [], '股票名称': [], '开始': [], '结果': [], '市场': [], '连续涨停数':[]}
    df_res = pd.DataFrame(list_industry)
    tmp_start = start_date
    # 涨停数
    count = 0
    for index, row in df_day.iterrows():
        if row['grow'] < market_max:
            break
        else:
            count += 1

        if count >= grow_up_count:
            tmp_start = row['trade_date']
            for k in list_industry.keys():
                list_industry[k].clear()
            list_industry['股票代码'].append(row['ts_code'])
            list_industry['股票名称'].append(name)
            list_industry['开始'].append(tmp_start)
            list_industry['结果'].append(row['grow'])
            list_industry['市场'].append(market)
            list_industry['连续涨停数'].append(count)
    df_res = pd.DataFrame(list_industry)
    if len(df_res) > 0:
        print(df_res)
    # print("%s : %s : %d" % (ts_code, name, df_res.shape[0]))
    return df_res


def get_single_data():
    df_all = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,name,market')
    list_industry = {'股票代码': [], '股票名称': [], '开始': [], '结果': [], '市场': [], '连续涨停数': []}
    df_res = pd.DataFrame(list_industry)
    size = df_all.shape[0]
    count = 1
    for index, row in df_all.iterrows():
        # print("%d : %s , %s  %d / %d" % (grow_up_count, row['ts_code'], row['name'], index, size))
        try:
           df_tmp = get_single_grow(row['ts_code'], row['name'], row['market'])
           df_res = df_res.append(df_tmp)
        except Exception as e:
           traceback.format_exc()
        if df_res.shape[0] > 1:
            count += 1
        # if count > 3:
        #     break
    tag = "推荐"
    df_res.to_csv('./res/%s.csv' % tag)
    print("*" * 50)
    print(tag)
    str_res = "总共出现： %d 次\n涨 %d 次， 跌 %d 次， 平 %d 次\n平均值： %f" % (df_res.shape[0],
                                                               df_res.loc[df_res["结果"] > 0].shape[0],
                                                               df_res.loc[df_res["结果"] < 0].shape[0],
                                                               df_res.loc[df_res["结果"] == 0].shape[0],
                                                               df_res['结果'].sum() / df_res.shape[0])
    print(str_res)
    file_path = "./res/%s.txt" % tag
    with open(file_path, "w+") as f:
        f.write(str_res)


get_single_data()

# df_res = get_single_grow("000856.SZ","aa","b")
# df_res

        股票代码 股票名称        开始       结果  市场  连续涨停数
0  000620.SZ  新华联  20220208  10.0694  主板      3
        股票代码  股票名称        开始     结果  市场  连续涨停数
0  000856.SZ  冀东装备  20220128  9.949  主板      5
        股票代码  股票名称        开始      结果   市场  连续涨停数
0  002104.SZ  恒宝股份  20220128  9.9483  中小板      5
        股票代码  股票名称        开始      结果   市场  连续涨停数
0  002530.SZ  金财互联  20220208  9.9907  中小板      3
        股票代码  股票名称        开始      结果   市场  连续涨停数
0  002663.SZ  普邦股份  20220208  9.8361  中小板      3
        股票代码  股票名称        开始      结果   市场  连续涨停数
0  002761.SZ  浙江建投  20220207  9.9617  中小板      4
        股票代码  股票名称        开始       结果   市场  连续涨停数
0  002878.SZ  元隆雅图  20220207  10.0108  中小板      4
        股票代码  股票名称        开始       结果  市场  连续涨停数
0  600939.SH  重庆建工  20220207  10.0737  主板      4
        股票代码  股票名称        开始      结果  市场  连续涨停数
0  603176.SH  汇通集团  20220207  10.011  主板      4
        股票代码  股票名称        开始     结果  市场  连续涨停数
0  603825.SH  华扬联众  20220208  9.995  主板      3
******************************